# Анализ NPS

Заказчик данного исследования — большая телекоммуникационная компания, которая оказывает услуги на территории всего СНГ. Перед компанией стоит задача определить текущий уровень потребительской лояльности, или NPS (от англ. Net Promoter Score), среди клиентов из России.

Компания провела опрос и попросила вас подготовить дашборд с его итогами. Большую базу данных для такой задачи разворачивать не стали и выгрузили данные в SQLite.

Чтобы оценить результаты опроса, оценки обычно делят на три группы:
- 9-10 баллов — «cторонники» (англ. promoters);
- 7-8 баллов — «нейтралы» (англ. passives);
- 0-6 баллов — «критики» (англ. detractors).

Итоговое значение NPS рассчитывается по формуле: % «сторонников» - % «критиков».

**Цель исследования** — ответить на следующие вопросы:

- Как распределены участники опроса по возрасту, полу и возрасту? Каких пользователей больше: новых или старых? Пользователи из каких городов активнее участвовали в опросе?

- Какие группы пользователей наиболее лояльны к сервису? Какие менее?

- Какой общий NPS среди всех опрошенных?

- Как можно описать клиентов, которые относятся к группе cторонников (англ. promoters)?

In [1]:
import pandas as pd
import numpy as np

from sqlalchemy import create_engine

In [2]:
path_to_db = '/datasets/telecomm_csi.db'
engine = create_engine(f'sqlite:///{path_to_db}', echo = False)

In [3]:
query = """
SELECT u.user_id,
       u.lt_day,
       CASE 
           WHEN u.lt_day <= 365 THEN 'Новый'
           ELSE 'Старый'
           END AS is_new,
       u.age,
       CASE 
           WHEN u.gender_segment == 1 THEN 'Женщина'
           WHEN u.gender_segment == 0 THEN 'Мужчина'
           ELSE 'z'
           END AS gender_segment,
       u.os_name,
       u.cpe_type_name,
       location.country,
       location.city,
       SUBSTR(age_segment.title, 4) AS age_segment,
       SUBSTR(traffic_segment.title, 4) AS traffic_segment,
       SUBSTR(lifetime_segment.title, 4) AS lifetime_segment,
       u.nps_score,
       CASE 
           WHEN u.nps_score >= 9 THEN 'Cторонник'
           WHEN u.nps_score >= 7 THEN 'Нейтрал'
           ELSE 'Критик'
           END AS nps_group
FROM user AS u
JOIN location ON u.location_id = location.location_id
JOIN age_segment ON u.age_gr_id = age_segment.age_gr_id
JOIN traffic_segment ON u.tr_gr_id = traffic_segment.tr_gr_id
JOIN lifetime_segment ON u.lt_gr_id = lifetime_segment.lt_gr_id;
"""

In [4]:
df = pd.read_sql(query, engine)
df.sample(10)

,user_id,lt_day,is_new,age,gender_segment,os_name,cpe_type_name,country,city,age_segment,traffic_segment,lifetime_segment,nps_score,nps_group
335792,RDLG5G,4611,Старый,51.0,Женщина,ANDROID,SMARTPHONE,Россия,Астрахань,45-54,1-5,36+,7,Нейтрал
86724,EGYUD3,655,Старый,39.0,Женщина,ANDROID,SMARTPHONE,Россия,Уфа,35-44,30-35,13-24,7,Нейтрал
396055,UHXMEQ,5313,Старый,59.0,Мужчина,ANDROID,SMARTPHONE,Россия,СанктПетербург,55-64,15-20,36+,10,Cторонник
498792,ZSUIYW,861,Старый,28.0,Мужчина,ANDROID,SMARTPHONE,Россия,Махачкала,25-34,10-15,25-36,10,Cторонник
178986,J8PXUZ,210,Новый,23.0,Женщина,IOS,SMARTPHONE,Россия,Казань,16-24,5-10,7-12,7,Нейтрал
471916,YF3E5Y,3013,Старый,32.0,Женщина,ANDROID,SMARTPHONE,Россия,Махачкала,25-34,10-15,36+,7,Нейтрал
249349,MW6ORU,4079,Старый,45.0,Женщина,ANDROID,SMARTPHONE,Россия,Оренбург,45-54,0.01-0.1,36+,10,Cторонник
192582,JY8UNZ,642,Старый,52.0,Женщина,ANDROID,SMARTPHONE,Россия,Ульяновск,45-54,0.1-1,13-24,10,Cторонник
211702,KXRE6B,1320,Старый,32.0,Женщина,ANDROID,SMARTPHONE,Россия,Саратов,25-34,0.1-1,36+,4,Критик
232264,M08AF7,5702,Старый,33.0,Мужчина,IOS,SMARTPHONE,Россия,Волгоград,25-34,35-40,36+,1,Критик


В таблице 14 столбцов:

- `user_id` — идентификатор клиента
- `lt_day` — количество дней «жизни» клиента
- `is_new` — поле хранит информацию о том, является ли клиент новым
- `age` — возраст
- `gender_segment` — пол
- `os_name` — тип операционной системы
- `cpe_type_name` — тип устройства
- `country` — страна проживания
- `city` — город проживания
- `age_segment` — возрастной сегмент
- `traffic_segment` — сегмент по объёму потребляемого трафика
- `lifetime_segment` — сегмент по количеству дней «жизни»
- `nps_score` — оценка клиента в NPS-опросе
- `nps_group` — поле хранит информацию о том, к какой группе относится оценка клиента в опросе

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 502493 entries, 0 to 502492
Data columns (total 14 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   user_id           502493 non-null  object 
 1   lt_day            502493 non-null  int64  
 2   is_new            502493 non-null  object 
 3   age               501939 non-null  float64
 4   gender_segment    502493 non-null  object 
 5   os_name           502493 non-null  object 
 6   cpe_type_name     502493 non-null  object 
 7   country           502493 non-null  object 
 8   city              502493 non-null  object 
 9   age_segment       502493 non-null  object 
 10  traffic_segment   502493 non-null  object 
 11  lifetime_segment  502493 non-null  object 
 12  nps_score         502493 non-null  int64  
 13  nps_group         502493 non-null  object 
dtypes: float64(1), int64(2), object(11)
memory usage: 53.7+ MB


In [6]:
df.to_csv('telecomm_csi_tableau.csv', index=False)

# Ссылка на дашборд вTableau Public:

https://public.tableau.com/app/profile/pochemuchto/viz/NPS_16489136364110/NPS_1

# Cсылка на pdf-файл с презентацией:

https://disk.yandex.ru/i/HM07ovO66NnKfA